# NER & SRL 실습
1. AllenNLP 라이브러리 사용
2. NER Predictor를 이용한 토큰 대체
3. SRL PRedictor를 이용한 스토리 이벤트 파싱

## 01 AllenNLP 라이브러리

In [ ]:
!pip install allennlp
!pip install allennlp-models

In [2]:
from allennlp.predictors.predictor import Predictor

In [ ]:
# NER과 SRL 모델 사용을 위한 predictor 정의를 해줍니다
ner_predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/ner-model-2020.02.10.tar.gz")
srl_predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz")

In [ ]:
srl_predictor.predict(sentence=documents[0])

In [ ]:
ner_predictor.predict(sentence=documents[0])

데이터 로드

In [ ]:
import pandas as pd

In [75]:
data_path = "/content/drive/MyDrive/05_강의자료/210719_SDS/ROCStories__spring2016 - ROCStories_spring2016.csv"

In [76]:
data = pd.read_csv(data_path)

In [77]:
data.head()

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5
0,9a51198e-96f1-42c3-b09d-a3e1e067d803,Overweight Kid,Dan's parents were overweight.,Dan was overweight as well.,The doctors told his parents it was unhealthy.,His parents understood and decided to make a c...,They got themselves and Dan on a diet.
1,617e7ada-3878-488d-bd56-40695b91f053,The Bike Accident,Carrie had just learned how to ride a bike.,She didn't have a bike of her own.,Carrie would sneak rides on her sister's bike.,She got nervous on a hill and crashed into a w...,The bike frame bent and Carrie got a deep gash...
2,79b0da1f-e460-4173-ba58-8c9e2553c53a,Beach,Morgan enjoyed long walks on the beach.,She and her boyfriend decided to go for a long...,"After walking for over a mile, something happe...",Morgan decided to propose to her boyfriend.,Her boyfriend was upset he didn't propose to h...
3,d173b7de-4611-4cdf-934c-912834755e41,The bad customer.,Jane was working at a diner.,"Suddenly, a customer barged up to the counter.",He began yelling about how long his food was t...,Jane didn't know how to react.,"Luckily, her coworker intervened and calmed th..."
4,af0fd5a4-de36-47ba-8aa2-e99d10986d7a,Being Patient,I was talking to my crush today.,She continued to complain about guys flirting ...,I decided to agree with what she says and list...,"After I got home, I got a text from her.",She asked if we can hang out tomorrow.


## 02 NER Predictor를 이용해 'U-PER' 토큰으로 대체하기

In [78]:
def ner_per_replace(sentence):
    ner_result = ner_predictor.predict(sentence=sentence)
    tags = ner_result['tags']
    words = ner_result['words']
    for i in range(len(tags)):
        if tags[i] == 'U-PER':
            words[i] = 'U-PER'
    sentence = ' '.join(words)
    return sentence

In [79]:
def story_per_replace(row):
    story = []
    columns = ['sentence1', 'sentence2', 'sentence3', 'sentence4', 'sentence5']
    for col in columns:
        story.append(ner_per_replace(row[col]))
    return story

In [80]:
story_per_replace(data.loc[0])

["U-PER 's parents were overweight .",
 'U-PER was overweight as well .',
 'The doctors told his parents it was unhealthy .',
 'His parents understood and decided to make a change .',
 'They got themselves and U-PER on a diet .']

## 03 SRL PRedictor를 이용해 동사별 argument 파싱하기

In [81]:
def srl_arg_replace(sentence):
    events = []
    srl_result = srl_predictor.predict(sentence=sentence)
    words = srl_result['words']
    for frame in srl_result['verbs']:
        events.append(frame['description'])
    return events

In [82]:
def story_event_parsing(row):
    story = []
    columns = ['sentence1', 'sentence2', 'sentence3', 'sentence4', 'sentence5']
    for col in columns:
        story.extend(srl_arg_replace(row[col]))
    return story

In [83]:
story_event_parsing(data.loc[0])

["[ARG1: Dan 's parents] [V: were] [ARG2: overweight] .",
 '[ARG1: Dan] [V: was] [ARG2: overweight] [ARGM-ADV: as well] .',
 '[ARG0: The doctors] [V: told] [ARG2: his parents] [ARG1: it was unhealthy] .',
 'The doctors told his parents [ARG1: it] [V: was] [ARG2: unhealthy] .',
 '[ARG0: His parents] [V: understood] and decided to make a change .',
 '[ARG0: His parents] understood and [V: decided] [ARG1: to make a change] .',
 '[ARG0: His parents] understood and decided to [V: make] [ARG1: a change] .',
 '[ARG0: They] [V: got] [ARG1: themselves and Dan] [ARG2: on a diet] .']